In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import numpy as np
from argparse import ArgumentParser
from double_net.double_net import DoubleNet, train_loop, train_loop_no_lagrange, test_loop
from double_net import datasets as ds
import time
import double_net.plot_utils as pu
import importlib
import matplotlib.pyplot as plt

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [3]:
device

'cuda'

In [4]:
parser = ArgumentParser()
parser.add_argument('--random-seed', type=int, default=0)
parser.add_argument('--num-examples', type=int, default=131072 * 4)
parser.add_argument('--test-num-examples', type=int, default=2048 * 8)
parser.add_argument('--test-iter', type=int, default=5)
parser.add_argument('--n-agents', type=int, default=1)
parser.add_argument('--n-items', type=int, default=2)
parser.add_argument('--num-epochs', type=int, default=250)
parser.add_argument('--batch-size', type=int, default=128)
parser.add_argument('--test-batch-size', type=int, default=4096)
parser.add_argument('--model-lr', type=float, default=1e-3)
parser.add_argument('--misreport-lr', type=float, default=1e-1)
parser.add_argument('--misreport-iter', type=int, default=25)
parser.add_argument('--test-misreport-iter', type=int, default=1000)
parser.add_argument('--rho', type=float, default=0.5)
parser.add_argument('--rho-incr-iter', type=int, default=10)
parser.add_argument('--rho-incr-amount', type=float, default=1)
parser.add_argument('--lagr-update-iter', type=int, default=11)
parser.add_argument('--rgt-start', type=int, default=0)
parser.add_argument('--sinkhorn-epsilon', type=float, default=5e-2)
parser.add_argument('--sinkhorn-rounds', type=int, default=30)

dataset_name = ["double_net/1x2-pv.csv"]
args = parser.parse_args(args=[])
args

Namespace(batch_size=128, lagr_update_iter=11, misreport_iter=25, misreport_lr=0.1, model_lr=0.001, n_agents=1, n_items=2, num_epochs=250, num_examples=524288, random_seed=0, rgt_start=0, rho=0.5, rho_incr_amount=1, rho_incr_iter=10, sinkhorn_epsilon=0.05, sinkhorn_rounds=30, test_batch_size=4096, test_iter=5, test_misreport_iter=1000, test_num_examples=16384)

In [5]:
torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)

item_ranges = ds.preset_valuation_range(args.n_agents, args.n_items, dataset_name)
clamp_op = ds.get_clamp_op(item_ranges)

model = DoubleNet(
    args.n_agents, args.n_items, clamp_op, args.sinkhorn_epsilon, args.sinkhorn_rounds
).to(device)

train_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.num_examples, item_ranges).to(device)
train_loader = ds.Dataloader(train_data, batch_size=16384 * 8, shuffle=True)

pavlov U[2,3] Auction


In [6]:
%%time
train_loop(model, train_loader, args, device=device)

  0%|          | 1/250 [00:03<15:12,  3.67s/it]

{'regret_max': 0.5441237688064575, 'regret_mean': 0.24253994226455688, 'payment': 1.4164798259735107}


  1%|          | 2/250 [00:07<15:05,  3.65s/it]

{'regret_max': 0.3198450803756714, 'regret_mean': 0.1374788135290146, 'payment': 1.3792366981506348}


  1%|          | 3/250 [00:10<14:58,  3.64s/it]

...

{'regret_max': 0.01063525676727295, 'regret_mean': 0.0012733928160741925, 'payment': 2.1551311016082764}


100%|█████████▉| 249/250 [15:12<00:03,  3.67s/it]

{'regret_max': 0.010806679725646973, 'regret_mean': 0.001295511145144701, 'payment': 2.1556429862976074}


100%|██████████| 250/250 [15:16<00:00,  3.67s/it]

{'regret_max': 0.010803461074829102, 'regret_mean': 0.0012933884281665087, 'payment': 2.1558170318603516}
CPU times: user 14min 11s, sys: 1min 2s, total: 15min 14s
Wall time: 15min 16s


In [7]:
%%time
test_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device)
test_loader = ds.Dataloader(test_data, batch_size=args.test_batch_size, shuffle=True)

# result = test_loop(model, test_loader, args, device=device)
# result

CPU times: user 999 µs, sys: 0 ns, total: 999 µs
Wall time: 1 ms


In [1]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100
model.sinkhorn_rounds = 300
model.sinkhorn_epsilon = 5e-2
pu.plot_12_model(model.to(device='cpu'), 0.01, (2,3), (2,3))

NameError: name 'plt' is not defined